# 네이버 자주 본 뉴스 크롤링

1) 수집내용

    1) 많이본뉴스–섹션별(정치~IT/과학)Top5기사제목,신문사,뷰 
    2) 해당 기사별 기사 내용, 리액션 (좋아요 ~ 후속기사 원해요)
    
2) 수집방법(택1)

    1) [기본] Requests , BeautifulSoup, Selenium
    2) [심화] Requests, BeautifulSoup (+ 멀티프로세싱)
    
3) 수집범위 및 저장

    1) 2019년7월21일~2020년8월20일(동작가능,실제구동x)
    2) 하나의 파일로 저장 (방식 자유)
    3) Ex)총6섹션*Top5*365일=10950rows

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from IPython.display import Image
import os
from pathlib import Path
import glob

import datetime

## 날짜 String 생성

In [2]:
start_date = datetime.datetime(2019, 7, 21, 3, 22, 32)
start_date.strftime("%Y%m%d %Y-%m-%d 시간 표시외 %H:%M:%S 아무거나 넣어도 됩니다.") #strftime 활용 예시

'20190721 2019-07-21 시간 표시외 03:22:32 아무거나 넣어도 됩니다.'

In [3]:
def get_date_string(start_date, period=365):
    # 20190721 형태의 Date String List 생성
    return [
        (start_date + datetime.timedelta(days=day)).strftime("%Y%m%d")
        for day in range(period)
    ]

In [4]:
date_string_list = get_date_string(start_date, 365)

In [5]:
date_string_list[0], date_string_list[-1] # 생성 완료

('20190721', '20200719')

## Part 1. Request를 이용한 크롤링

In [6]:
def get_top_news(date):
    title_text=[] #기사제목
    url_text = [] #기사링크
    source_text=[] #신문사, 방송사 이름
    view_text=[] #뷰 
    """
    해당 날짜의 자주 본 뉴스 25개에 대한 정보를 반환하는 함수입니다.
    """
    url = f"https://news.naver.com/main/ranking/popularDay.nhn?rankingType=popular_day&date={date}"
    res = requests.get(url)
    
    #뷰티풀소프의 인자값 지정
    soup = BeautifulSoup(res.text, "html.parser")
    
    result={} #결과를 뽑기 위한 딕셔너리 생성
 
    # soup.select를 잘 활용하여
    # 1. 신문 제목
    # 2. 기사 링크 (a tag의 href 속성)
    titles = soup.select('.ranking_section > ol > li > dl > dt > a') 
    for i in range(len(titles)):
        title_text.append(titles[i].text)
        url_text.append(titles[i]['href'])
     
    # 3. 신문사명
    for dl in soup.select("dl"):
        source_text.append(dl.select("span")[-2].text.strip())
        
    views = soup.select('.count_view') 
    for i in range(len(views)):
        view_text.append(views[i].text) 
    del view_text[30:] #포토/TV 섹션에 있는 뷰까지도 다들어가서 삭제(df만들때 개수맞추기위함)
        
    result= {"title":title_text , "url":url_text, "source" : source_text ,"views": view_text }
    df = pd.DataFrame(result) #데이터프레임으로 변환
    
    return df

In [7]:
TEST_DATE_RANGE = 10 # 1년치 전체를 하는 것이 아닌 앞 20일치만 테스트 해보기 위함 (개인적으로 바꿔도 무방)

news_data = []
for date in date_string_list[:TEST_DATE_RANGE]:
    news_data.append(get_top_news(date)) # 결과로 나온 25개를 이어 붙임

In [8]:
df_top_news = pd.concat(news_data).reset_index(drop=True)
#news_data로 원래 index는 지워주고 합쳐준다!

In [9]:
df_top_news

,title,url,source,views
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,경향신문,"201,039"
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,머니투데이,"185,396"
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,연합뉴스,"130,198"
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,동아일보,"120,897"
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,조선일보,"119,463"
...,...,...,...,...
295,"박경락, 전 프로게이머 사망…임요환·홍진호 어깨 나란히",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,한국경제,"214,190"
296,의류 건조기 이물질 논란…그래서 4종을 직접 뜯어봤다,/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,전자신문,"115,164"
297,"[속보] LG전자, 2분기 영업익 6523억…전년비 15% '감소'",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,한국경제,"84,179"
298,"한국, 전자산업 생산 일본 제치고 세계 3위",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,전자신문,"58,010"


In [10]:
# URL 앞에  "https://news.naver.com" 붙이기
df_top_news.url = "https://news.naver.com"+df_top_news.url

In [11]:
df_top_news

,title,url,source,views
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",https://news.naver.com/main/ranking/read.nhn?m...,경향신문,"201,039"
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",https://news.naver.com/main/ranking/read.nhn?m...,머니투데이,"185,396"
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",https://news.naver.com/main/ranking/read.nhn?m...,연합뉴스,"130,198"
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,https://news.naver.com/main/ranking/read.nhn?m...,동아일보,"120,897"
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"119,463"
...,...,...,...,...
295,"박경락, 전 프로게이머 사망…임요환·홍진호 어깨 나란히",https://news.naver.com/main/ranking/read.nhn?m...,한국경제,"214,190"
296,의류 건조기 이물질 논란…그래서 4종을 직접 뜯어봤다,https://news.naver.com/main/ranking/read.nhn?m...,전자신문,"115,164"
297,"[속보] LG전자, 2분기 영업익 6523억…전년비 15% '감소'",https://news.naver.com/main/ranking/read.nhn?m...,한국경제,"84,179"
298,"한국, 전자산업 생산 일본 제치고 세계 3위",https://news.naver.com/main/ranking/read.nhn?m...,전자신문,"58,010"


## Part 2. Selenium을 이용한 크롤링

In [12]:
driver_path =  os.path.join(Path().absolute(),  "chromedriver.exe")

In [13]:
driver = webdriver.Chrome(driver_path) # Chrome driver 실행

In [14]:
df_top_news.url[:10]

0    https://news.naver.com/main/ranking/read.nhn?m...
1    https://news.naver.com/main/ranking/read.nhn?m...
2    https://news.naver.com/main/ranking/read.nhn?m...
3    https://news.naver.com/main/ranking/read.nhn?m...
4    https://news.naver.com/main/ranking/read.nhn?m...
5    https://news.naver.com/main/ranking/read.nhn?m...
6    https://news.naver.com/main/ranking/read.nhn?m...
7    https://news.naver.com/main/ranking/read.nhn?m...
8    https://news.naver.com/main/ranking/read.nhn?m...
9    https://news.naver.com/main/ranking/read.nhn?m...
Name: url, dtype: object

In [16]:
import time
NEWS_TEST_RANGE = 10
for idx, news_url in enumerate(df_top_news.url[:NEWS_TEST_RANGE]):
    # 드라이버 내에서 해당 URL로 이동
    driver.get(news_url)
    time.sleep(3) 
    soup = BeautifulSoup(driver.page_source,"html.parser")
    
    content = []
    contets = soup.select('#articleBodyContents')
    for i in range(len(contets)):
        content.append(contets[i].text.strip()) # Content 앞 데이터 전처리
        
    like =[]
    warm = []
    sad = []
    angry = []
    want = []
    #리스트에 담지않고 바로 가져오는방법
    #df_top_news.loc[idx,"like"] = ul.select_one("li.u_likeit_list.sad > a > span.u_likeit_list_count._count").text 
 
    
    react = soup.select('.end_btn > div > ul')
    for ul in react:
        
        like.append(ul.select_one("li.u_likeit_list.good > a > span.u_likeit_list_count._count").text)
        warm.append(ul.select_one("li.u_likeit_list.warm > a > span.u_likeit_list_count._count").text)
        sad.append(ul.select_one("li.u_likeit_list.sad > a > span.u_likeit_list_count._count").text)
        angry.append(ul.select_one("li.u_likeit_list.angry > a > span.u_likeit_list_count._count").text)
        want.append(ul.select_one("li.u_likeit_list.want > a > span.u_likeit_list_count._count").text)
        

    #하나하나 써본거를 위에 for문으로 합쳐보았다.
    #r_like = soup.select_one('ul > li.u_likeit_list.good > a > span.u_likeit_list_count._count').text
    #like.append(r_like)
    
   
    #r_warm = soup.select_one('ul > li.u_likeit_list.warm > a > span.u_likeit_list_count._count').text
    #warm.append(r_warm)
    
    
    #r_sad = soup.select_one(' ul > li.u_likeit_list.sad > a > span.u_likeit_list_count._count').text
    #sad.append(r_sad)
    
   
    #r_angry = soup.select_one('ul > li.u_likeit_list.angry > a > span.u_likeit_list_count._count').text
    #angry.append(r_angry)
    
    
    #r_want = soup.select_one('ul > li.u_likeit_list.want > a > span.u_likeit_list_count._count').text
    #want.append(r_want)
    
    
    
    # 예시로 content라는 변수에 기사 내용을 담고 Column "content"에 해당 내용 저장
    df_top_news.loc[idx,"content"] = content
    df_top_news.loc[idx,"like"] = like
    df_top_news.loc[idx,"warm"] = warm
    df_top_news.loc[idx,"sad"] = sad
    df_top_news.loc[idx,"angry"] = angry
    df_top_news.loc[idx,"want"] = want
    

In [18]:
df_top_news.head()

,title,url,source,views,content,like,warm,sad,angry,want
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",https://news.naver.com/main/ranking/read.nhn?m...,경향신문,"201,039",중 3 때 오빠와 장관상이어 고3 때도 ‘장함모’ 활동으로 자원봉사대회 금상“대체로...,131,16,17,"2,605",121
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",https://news.naver.com/main/ranking/read.nhn?m...,머니투데이,"185,396","[[머니투데이 김성휘 ,백지수 기자] [[the300]징용판결 해설 글 ""대통령 ...","[1,710]",[32],[13],"[6,058]",[48]
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",https://news.naver.com/main/ranking/read.nhn?m...,연합뉴스,"130,198","[""文정부, 서희와 이순신 역할 함께 수행…지레 겁먹지 말아야""""文정부 매도 정치인...","[1,799]",[19],[13],"[9,120]",[30]
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,https://news.naver.com/main/ranking/read.nhn?m...,동아일보,"120,897","[친일잔재를 청산하고 한번도 경험하지 못한 나라로 가는 것이 목적이라면, 문재인 정...","[3,017]",[19],[16],[623],[37]
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"119,463","[""문재인 정부, 국익 수호 위해 '서희' '이순신' 역할 함께 수행""""법적·외교적...",[374],[13],[14],"[11,468]",[38]


> 본 데이터 프레임은 예시로, 리액션 데이터를 담지 않았습니다. **과제에는 리액션 데이터가 반드시 담겨져있어야합니다.**

# 데이터 저장

> 파일 형태로 크롤링한 데이터를 저장

In [ ]:
#RESULT_PATH ='./' 
#FileName = "naver_top_news.xlsx"
#df_top_news.to_excel(RESULT_PATH+FileName, sheet_name='sheet1')